## Server context

> **Nginx** allows for **multiple virtual servers** to be specified (more like apache), which are defined in the config files under the following directory,
```bash
/etc/nginx/conf.d/
```
> Recall that this directory was included in the main **nginx.conf** file
```bash
 include /etc/nginx/conf.d/*.conf;
```
> Now, we ll look at how we can define our own servers

### Multiple servers in same machine

> 1) **Purchase** domain names to make it available in the DNS

> 2) Remember that **multiple domain names can be mapped to single ip address**

> 3) **Njinx** can **resolve** multiple domain names in same machine

> 4) **Domain names** in **DNS** are only mapped to **public ip**, which is **ip of the modem**. To expose the server in your local machine in network, you have to **forward port 80** (note that you cant forward a single port to multiple machines in a network) 

> 5) For debugging, you dont need to purchase a domain name. Instead map the domain name in **/etc/hosts** of the **host machine** (not inside container). When you open the browser in the host machine, it will first try to resolve from **/etc/hosts/** before moving to DNS 

>> **NOTE**: If you map a domain name from DNS in **/etc/hosts**, sometimes (not always) it will **override** the actual domain and you will not be able to access that URL

<img src="files/domainmap.png" />

> 6) When you open the browser (remember that the browser is outside the container) and hit **d1.com** or **d2.com**, it resolves the domain name from **DNS** and **forwards** the request to **port 80** of the **container**, where **nginx** then resolves to the required location. 

> 7) When you hot **tf.com**, the browser resolves the domain in **/etc/hosts** and forwards to port 80 of the container. Note that you can access **tf.com** only from the same machine in which the container exists.

## Demonstration showing deployment of three servers

> **UPDATE to docker-run command**

>> Map the following volumes

```
        -v ${PWD}/html/:/usr/share/nginx/html \
        -v ${PWD}/confs/nginx.conf:/etc/nginx/nginx.conf \
        -v ${PWD}/confs/conf.d/:/etc/nginx/conf.d \
```

> **Create html content for each server**

        html/
            d1/
                index.html
            d2/
                index.html
            wordpress/
                index.html

> **conf.d/servers.conf** (make sure to include this in **nginx.conf**)

<div style="border:2px solid;margin:0 20px 0 20px"></div>

```
        server {
          listen       80;
          server_name  www.d1.example.com  d1.com;

          location / {
              root   /usr/share/nginx/html/d1;
          }
        }

        server {
          listen       80;
          server_name  www.d2.example.com;

          location / {
              root   /usr/share/nginx/html/d2;
          }
        }

        server {
          listen       80;
          server_name  www.wordpress.com;

          location / {
              root   /usr/share/nginx/html/wordpress;
          }
        }
```
<div style="border:2px solid;margin:0 20px 0 20px"></div>


> **Map the domain names in /etc/hosts of the host machine** (not inside docker)
```bash
192.168.1.2     www.d1.example.com  d1.com
192.168.1.2     www.d2.example.com
# this will override original website
192.168.1.2     www.wordpress.com 
```

> **Now your browser will the above urls to their respective index.html**

>> **Default server**: If you just hit **localhost** or 192.168.1.2, it will resolve to the **first server** (when you dont have a default.conf). Otherwise you can specify a **default server** which will give a **page not found response**
```python
# this is used when no server name matches
# this is also used when we hit localhost (its not recommended to make server accessible with ip)
server {
    listen      80 default_server;
    server_name _ ;
    return 404;
}
```

>> Now the client will be able to access the server only with the right name